## Get the data

Real-world data tends to occupy relational databases and Amazon S3 buckets. You better brush off your SQL and file processing skills. Luckily, languages like Python have a wide variety of helpers (both built-in and as external packages).

Working with raw files require an understanding of how text and image files are stored, can be loaded and shown.

### Start with a sample

Get 10 or 100 examples. Try to get somewhat of a representative subset of the data, but don't try too hard - a simple random subset will do (for now).

### Request more data

Some additional data can be easily stored for you. How easy would it be to get that done? Can someone else do it for you?

## Labelling

Don't have enough or no data at all? You can create your own datasets! Yes, but that is not an easy task.

## Look at the data

## Is the data clean?

## Splitting

## Versioning